In [6]:
import os
import time
from pynq import Overlay, allocate, MMIO
from PIL import Image
import numpy as np

# 0) List of your 10 input images (adjust names as needed)
input_images = [
    'image1.jpg', 'image2.jpg', 'image3.jpg', 'image4.jpg', 'image5.jpg',
    'image6.jpg', 'image7.jpg', 'image8.jpg', 'image9.jpg', 'image10.jpg'
]

# 1) Load the bitstream & set up MMIO
ol   = Overlay('convolution.bit')
base = ol.Convolution_0.mmio.base_addr
mm   = MMIO(base, ol.Convolution_0.mmio.length)
print(f"MMIO @ 0x{base:08X}")

# 2) Register offsets
REG_CTRL      = 0x00
REG_IMG_LO    = 0x10
REG_IMG_HI    = 0x14
REG_OUT_LO    = 0x1C
REG_OUT_HI    = 0x20
REG_WIDTH     = 0x28
REG_HEIGHT    = 0x30
REG_CHANNELS  = 0x38

# 3) Make sure output folder exists
output_folder = 'Convoluted images'
os.makedirs(output_folder, exist_ok=True)

start_time = time.time()

# Process each image in turn
for idx, img_name in enumerate(input_images, start=1):
    print(f"\n--- Processing {img_name} (#{idx}) ---")
    # 3a) Load & convert
    img = Image.open(img_name).convert('RGB')
    w, h = img.size
    data = np.array(img, dtype=np.uint8)

    # 3b) Allocate buffers
    in_buf  = allocate((h, w, 3), dtype=np.uint8)
    out_buf = allocate((h, w, 3), dtype=np.uint8)
    in_buf[:] = data

    # 4) Program dimensions & pointers
    mm.write(REG_WIDTH,    w)
    mm.write(REG_HEIGHT,   h)
    mm.write(REG_CHANNELS, 3)

    # Input buffer address
    lo = in_buf.physical_address & 0xFFFFFFFF
    hi = (in_buf.physical_address >> 32) & 0xFFFFFFFF
    mm.write(REG_IMG_LO, lo)
    mm.write(REG_IMG_HI, hi)

    # Output buffer address
    lo = out_buf.physical_address & 0xFFFFFFFF
    hi = (out_buf.physical_address >> 32) & 0xFFFFFFFF
    mm.write(REG_OUT_LO, lo)
    mm.write(REG_OUT_HI, hi)

    # 5) Start the IP
    mm.write(REG_CTRL, 1)

    # 6) Poll for completion
    while not (mm.read(REG_CTRL) & 0x2):
        time.sleep(1e-4)

    # 7) Retrieve & save the result
    result = out_buf.reshape((h, w, 3))
    out_path = os.path.join(output_folder, f'convolution_output_{idx}.jpg')
    Image.fromarray(result, 'RGB').save(out_path)
    print(f"Saved: {out_path}")

end_time = time.time()
elapsed = end_time - start_time
print(f"\nAll done! Total convolution time: {elapsed:.3f} seconds")


MMIO @ 0x40000000

--- Processing image1.jpg (#1) ---
Saved: Convoluted images/convolution_output_1.jpg

--- Processing image2.jpg (#2) ---
Saved: Convoluted images/convolution_output_2.jpg

--- Processing image3.jpg (#3) ---
Saved: Convoluted images/convolution_output_3.jpg

--- Processing image4.jpg (#4) ---
Saved: Convoluted images/convolution_output_4.jpg

--- Processing image5.jpg (#5) ---
Saved: Convoluted images/convolution_output_5.jpg

--- Processing image6.jpg (#6) ---
Saved: Convoluted images/convolution_output_6.jpg

--- Processing image7.jpg (#7) ---
Saved: Convoluted images/convolution_output_7.jpg

--- Processing image8.jpg (#8) ---
Saved: Convoluted images/convolution_output_8.jpg

--- Processing image9.jpg (#9) ---
Saved: Convoluted images/convolution_output_9.jpg

--- Processing image10.jpg (#10) ---
Saved: Convoluted images/convolution_output_10.jpg

All done! Total convolution time: 420.865 seconds


In [7]:
import os
import time
from pynq import Overlay, allocate, MMIO
from PIL import Image
import numpy as np

# 0) List of your 10 input images (adjust names as needed)
input_images = [
    'image1.jpg', 'image2.jpg', 'image3.jpg', 'image4.jpg', 'image5.jpg',
    'image6.jpg', 'image7.jpg', 'image8.jpg', 'image9.jpg', 'image10.jpg'
]

# 1) Load the bitstream & set up MMIO
ol   = Overlay('maxpooling.bit')
base = ol.MaxPooling_0.mmio.base_addr
mm   = MMIO(base, ol.MaxPooling_0.mmio.length)
print(f"MMIO @ 0x{base:08X}")

# 2) Register offsets
REG_CTRL      = 0x00
REG_IMG_LO    = 0x10
REG_IMG_HI    = 0x14
REG_OUT_LO    = 0x1C
REG_OUT_HI    = 0x20
REG_WIDTH     = 0x28
REG_HEIGHT    = 0x30
REG_CHANNELS  = 0x38

start_time = time.time()
# 3) Make sure output folder exists
output_folder = 'Max pooled images'
os.makedirs(output_folder, exist_ok=True)

start_time = time.time()
# Process each image in turn
for idx, img_name in enumerate(input_images, start=1):
    print(f"\n--- Processing {img_name} (#{idx}) ---")
    # 3a) Load & convert
    img = Image.open(img_name).convert('RGB')
    w, h = img.size
    data = np.array(img, dtype=np.uint8)

    # 3b) Allocate buffers
    in_buf  = allocate((h, w, 3), dtype=np.uint8)
    out_buf = allocate((h // 2, w // 2, 3), dtype=np.uint8)
    in_buf[:] = data

    # 4) Program dimensions & pointers
    mm.write(REG_WIDTH,    w)
    mm.write(REG_HEIGHT,   h)
    mm.write(REG_CHANNELS, 3)

    # Input buffer address
    lo = in_buf.physical_address & 0xFFFFFFFF
    hi = (in_buf.physical_address >> 32) & 0xFFFFFFFF
    mm.write(REG_IMG_LO, lo)
    mm.write(REG_IMG_HI, hi)

    # Output buffer address
    lo = out_buf.physical_address & 0xFFFFFFFF
    hi = (out_buf.physical_address >> 32) & 0xFFFFFFFF
    mm.write(REG_OUT_LO, lo)
    mm.write(REG_OUT_HI, hi)

    # 5) Start the IP
    mm.write(REG_CTRL, 1)

    # 6) Poll for completion
    while not (mm.read(REG_CTRL) & 0x2):
        time.sleep(1e-4)

    # 7) Retrieve & save the result
    result = out_buf.reshape((h // 2, w // 2, 3))
    out_path = os.path.join(output_folder, f'max_pooled_output_{idx}.jpg')
    Image.fromarray(result, 'RGB').save(out_path)
    print(f"Saved: {out_path}")

end_time = time.time()
elapsed = end_time - start_time
print(f"\nAll done! Total pooling time: {elapsed:.3f} seconds")


MMIO @ 0x40000000

--- Processing image1.jpg (#1) ---
Saved: Max pooled images/max_pooled_output_1.jpg

--- Processing image2.jpg (#2) ---
Saved: Max pooled images/max_pooled_output_2.jpg

--- Processing image3.jpg (#3) ---
Saved: Max pooled images/max_pooled_output_3.jpg

--- Processing image4.jpg (#4) ---
Saved: Max pooled images/max_pooled_output_4.jpg

--- Processing image5.jpg (#5) ---
Saved: Max pooled images/max_pooled_output_5.jpg

--- Processing image6.jpg (#6) ---
Saved: Max pooled images/max_pooled_output_6.jpg

--- Processing image7.jpg (#7) ---


KeyboardInterrupt: 

In [8]:
import os
import time
from pynq import Overlay, allocate, MMIO
from PIL import Image
import numpy as np

# 0) List of your 10 input images (adjust names as needed)
input_images = [
    'image1.jpg', 'image2.jpg', 'image3.jpg', 'image4.jpg', 'image5.jpg',
    'image6.jpg', 'image7.jpg', 'image8.jpg', 'image9.jpg', 'image10.jpg'
]

# 1) Load the bitstream & set up MMIO
ol   = Overlay('minpooling.bit')
base = ol.min_pooling_0.mmio.base_addr
mm   = MMIO(base, ol.min_pooling_0.mmio.length)
print(f"MMIO @ 0x{base:08X}")

# 2) Register offsets
REG_CTRL      = 0x00
REG_IMG_LO    = 0x10
REG_IMG_HI    = 0x14
REG_OUT_LO    = 0x1C
REG_OUT_HI    = 0x20
REG_WIDTH     = 0x28
REG_HEIGHT    = 0x30
REG_CHANNELS  = 0x38

# 3) Make sure output folder exists
output_folder = 'Min pooled images'
os.makedirs(output_folder, exist_ok=True)

# Process each image in turn
for idx, img_name in enumerate(input_images, start=1):
    print(f"\n--- Processing {img_name} (#{idx}) ---")
    # 3a) Load & convert
    img = Image.open(img_name).convert('RGB')
    w, h = img.size
    data = np.array(img, dtype=np.uint8)

    # 3b) Allocate buffers
    in_buf  = allocate((h, w, 3), dtype=np.uint8)
    out_buf = allocate((h // 2, w // 2, 3), dtype=np.uint8)
    in_buf[:] = data

    # 4) Program dimensions & pointers
    mm.write(REG_WIDTH,    w)
    mm.write(REG_HEIGHT,   h)
    mm.write(REG_CHANNELS, 3)

    # Input buffer address
    lo = in_buf.physical_address & 0xFFFFFFFF
    hi = (in_buf.physical_address >> 32) & 0xFFFFFFFF
    mm.write(REG_IMG_LO, lo)
    mm.write(REG_IMG_HI, hi)

    # Output buffer address
    lo = out_buf.physical_address & 0xFFFFFFFF
    hi = (out_buf.physical_address >> 32) & 0xFFFFFFFF
    mm.write(REG_OUT_LO, lo)
    mm.write(REG_OUT_HI, hi)

    # 5) Start the IP
    mm.write(REG_CTRL, 1)

    # 6) Poll for completion
    while not (mm.read(REG_CTRL) & 0x2):
        time.sleep(1e-4)

    # 7) Retrieve & save the result
    result = out_buf.reshape((h // 2, w // 2, 3))
    out_path = os.path.join(output_folder, f'min_pooled_output_{idx}.jpg')
    Image.fromarray(result, 'RGB').save(out_path)
    print(f"Saved: {out_path}")

end_time = time.time()
elapsed = end_time - start_time
print(f"\nAll done! Total pooling time: {elapsed:.3f} seconds")



MMIO @ 0x40000000

--- Processing image1.jpg (#1) ---
Saved: Min pooled images/min_pooled_output_1.jpg

--- Processing image2.jpg (#2) ---
Saved: Min pooled images/min_pooled_output_2.jpg

--- Processing image3.jpg (#3) ---
Saved: Min pooled images/min_pooled_output_3.jpg

--- Processing image4.jpg (#4) ---
Saved: Min pooled images/min_pooled_output_4.jpg

--- Processing image5.jpg (#5) ---
Saved: Min pooled images/min_pooled_output_5.jpg

--- Processing image6.jpg (#6) ---
Saved: Min pooled images/min_pooled_output_6.jpg

--- Processing image7.jpg (#7) ---
Saved: Min pooled images/min_pooled_output_7.jpg

--- Processing image8.jpg (#8) ---
Saved: Min pooled images/min_pooled_output_8.jpg

--- Processing image9.jpg (#9) ---
Saved: Min pooled images/min_pooled_output_9.jpg

--- Processing image10.jpg (#10) ---
Saved: Min pooled images/min_pooled_output_10.jpg

All done! Total pooling time: 74.651 seconds
